In [ ]:
# Install latest transformers for Gemma 3N
# !pip install --no-deps transformers>=4.53.1 # Only for Gemma 3N
# !pip install --no-deps --upgrade timm # Only for Gemma 3N
# from huggingface_hub import login
# login()

In [39]:
# Get the model from Unsloth
from unsloth import FastModel
import torch

torch._dynamo.config.cache_size_limit = 64  # or higher  

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 2048, # Used by Lucas
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, 
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.55.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
# Get the finetuning model
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [41]:
# Get the chat template
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3n",
)

# Use this chat template for training tool calls
tokenizer.chat_template = (
    "{{ bos_token }}{% for message in messages %}{% if message['role'] != 'system' %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"
)

In [4]:
# Load the dataset
from datasets import load_dataset
dataset = load_dataset("lmassaron/hermes-function-calling-v1", split = "train")
dataset["conversations"][0]

[{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'search_book', 'description': 'Search for a book based on title and/or author', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the book'}, 'author': {'type': 'string', 'description': 'The author of the book'}}, 'required': []}}}, {'type': 'function', 'function': {'name': 'get_definition', 'description': 'Get the definition of a word', 'parameters': {'type': 'object', 'properties': {'word': {'type': 'string', 'description': 'The word to get the definition for'}}, 'required': ['word']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 't

In [5]:
# Convert the dataset to the correct format for finetuning
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[100]["text"]

"<start_of_turn>human\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'analyze_website', 'description': 'Analyze the content and structure of a website', 'parameters': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the website to analyze'}}, 'required': ['url']}}}, {'type': 'function', 'function': {'name': 'calculate_bmi', 'description': 'Calculate the Body Mass Index (BMI)', 'parameters': {'type': 'object', 'properties': {'weight': {'type': 'number', 'description': 'The weight in kilograms'}, 'height': {'type': 'number', 'description': 'The height in meters'}}, 'required': ['weight', 'height']}}}] </tools>Use the following pydantic model json schema for each tool call you will

In [ ]:
# These training arguments were used by Lucas but we won't use them for now
lora_arguments = {
    "r": 16,
    "lora_alpha": 64,
    "lora_dropout": 0.05,
    "target_modules": [
        "embed_tokens",
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "o_proj",
        "lm_head",
    ],
}

training_arguments = {
    # Basic training configuration
    #"num_train_epochs": 1,
    "max_steps": 120,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 4,
    "max_seq_length": 2048,
    "packing": True,
    # Optimization settings
    "optim": "adamw_torch_fused",
    "learning_rate": 1e-4,
    "weight_decay": 0.1,
    "max_grad_norm": 1.0,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    # Memory optimization
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs": {"use_reentrant": False},
    # Evaluation and saving
    "eval_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 2,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_loss",
    "greater_is_better": False,
    # Logging and output
    "logging_steps": 5,
    "report_to": None,
    "logging_dir": "logs/runs",
    "overwrite_output_dir": True,
    # Model sharing
    "push_to_hub": False,
    "hub_private_repo": False,
}


In [52]:
# Setup the fine-tuning trainer
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 120,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

In [13]:
# Train the model on the responses only, ignore user instructions
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [14]:
# Verify the chat template was applied correctly. Only 1 <bos> token should be present.
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>human\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'analyze_website', 'description': 'Analyze the content and structure of a website', 'parameters': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the website to analyze'}}, 'required': ['url']}}}, {'type': 'function', 'function': {'name': 'calculate_bmi', 'description': 'Calculate the Body Mass Index (BMI)', 'parameters': {'type': 'object', 'properties': {'weight': {'type': 'number', 'description': 'The weight in kilograms'}, 'height': {'type': 'number', 'description': 'The height in meters'}}, 'required': ['weight', 'height']}}}] </tools>Use the following pydantic model json schema for each tool call you

In [15]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                                                                                                                                                                                                                                                                                                                                                                    Of course, I can help you with that. Could you please provide me with the URL of the website you want to analyze?<end_of_turn><eos>\n<start_of_turn>human\nSure, the website URL is www.example.com.<end_of_turn><eos>\n<start_of_turn>model\n<tool_call>\n{'name': 'analyze_website', 'arguments': {'url': 'www.example.com'}}\n</tool_call><end_of_turn><eos>\n<start_of_turn>tool\n<tool_response>\n{'status': 'success', 'message': 'Website analysis completed', 'data': {'structure': 'The website has a clear and intuitive structure with a navigation menu at the top. The homepage, about us, services, a

In [53]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,167 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 19,210,240 of 7,869,188,432 (0.24% trained)


Step,Training Loss
1,8.997900
2,8.941900
3,4.707700
4,4.441200
5,3.195000
6,3.475600
7,2.752500
8,3.477500
9,2.796500
10,2.181600


In [ ]:
model_name = "gemma3n_e4b_tools_test"
model_merged_hf_repo = "allisterb/gemma3n_e4b_tools_test"
model_gguf_hf_repo = model_merged_hf_repo + "-GGUF"

In [ ]:
# Save the model and tokenizer
model.save_pretrained(model_name)  # Local saving
tokenizer.save_pretrained(model_name)

['gemma3n_e4b_tools_test/processor_config.json']

In [26]:
# Save to float16
model.save_pretrained_merged(model_name, tokenizer)

Found HuggingFace hub cache directory: /home/eddsa-key-20250707/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  25%|██▌       | 1/4 [00:40<02:02, 40.89s/it]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 2/4 [01:56<02:02, 61.37s/it]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [03:11<01:07, 67.72s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [03:46<00:00, 56.55s/it]


In [96]:
model.save_pretrained_gguf(model_name, "Q8_0", tokenizer) 

Unsloth GGUF:hf-to-gguf:Loading model: gemma3n_e4b_tools_test
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3nForConditionalGeneration
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
Unsloth GGUF:hf-to-gguf:altup_proj.weight,                 torch.bfloat16 --> Q8_0, shape = {2048, 2048, 3}
Unsloth GGUF:hf-to-gguf:altup_unembd_proj.weight,          torch.bfloat16 --> Q8_0, shape = {2048, 2048, 3}
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {2048, 262144}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00004.safetensors'
Unsloth GGUF:hf-to-gguf:per_layer_token_embd.weight,       torch.bfloat16 --> Q8_0, shape = {8960, 262144}
Unsloth GGUF:hf-to-gguf:output_norm.weight,          

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to gemma3n_e4b_tools_test.Q8_0.gguf with size = 7.3G
Unsloth: Successfully saved GGUF to:
gemma3n_e4b_tools_test.Q8_0.gguf


['gemma3n_e4b_tools_test.Q8_0.gguf']

In [ ]:
# Push merged model to Hugging Face Hub
model.push_to_hub_gguf(model_name, "Q8_0", repo_id = model_gguf_hf_repo)

Unsloth GGUF:hf-to-gguf:Loading model: gemma3n_e4b_tools_test
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3nForConditionalGeneration
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
Unsloth GGUF:hf-to-gguf:altup_proj.weight,                 torch.bfloat16 --> Q8_0, shape = {2048, 2048, 3}
Unsloth GGUF:hf-to-gguf:altup_unembd_proj.weight,          torch.bfloat16 --> Q8_0, shape = {2048, 2048, 3}
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {2048, 262144}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00004.safetensors'
Unsloth GGUF:hf-to-gguf:per_layer_token_embd.weight,       torch.bfloat16 --> Q8_0, shape = {8960, 262144}
Unsloth GGUF:hf-to-gguf:output_norm.weight,          

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to gemma3n_e4b_tools_test.Q8_0.gguf with size = 7.3G
Unsloth: Successfully saved GGUF to:
gemma3n_e4b_tools_test.Q8_0.gguf


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


['gemma3n_e4b_tools_test.Q8_0.gguf']

In [ ]:
# Use if problems pushing
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="gemma3n_e4b_tools_test.Q8_0.gguf",
    path_in_repo="gemma3n_e4b_tools_test.Q8_0.gguf",
    repo_id=model_gguf_hf_repo
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  gemma3n_e4b_tools_test.Q8_0.gguf      :   0%|          | 33.5MB / 7.30GB            

CommitInfo(commit_url='https://huggingface.co/allisterb/gemma3n_e4b_tools_test-GGUF/commit/25c3a42ccf850e99d108249dbce419c534bbb322', commit_message='Upload gemma3n_e4b_tools_test.Q8_0.gguf with huggingface_hub', commit_description='', oid='25c3a42ccf850e99d108249dbce419c534bbb322', pr_url=None, repo_url=RepoUrl('https://huggingface.co/allisterb/gemma3n_e4b_tools_test-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='allisterb/gemma3n_e4b_tools_test-GGUF'), pr_revision=None, pr_num=None)